In [1]:
import yaml
import fitz
import torch
import gradio as gr
from PIL import Image
from langchain import hub
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
#from langchain_community.llms import HuggingFacePipeline
#from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import PyPDFLoader, PDFMinerLoader
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter,NLTKTextSplitter
from langchain_core.runnables import RunnableParallel
from langchain.retrievers import BM25Retriever, EnsembleRetriever
#TEST
from FakeLLM import FakePromptCopyLLM
import re  # For regular expressions
from nltk.corpus import stopwords  # You'll need to install NLTK first
import copy


d:\machineLearning\miniconda\envs\env_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
pdf_loader = PDFMinerLoader("../documents/2312.10997.pdf", concatenate_pages = False)
documents = pdf_loader.load()
def clean_text(text):
  #text = text.lower()  # Convert to lowercase
  #text = re.sub(r'[^a-z0-9\s-]', '', text)  # Remove non-alphanumeric characters (except space and dash)
  text = re.sub(r'-\n', '', text)  # Remove hyphens at line breaks
  return text
def preprocess_documents(documents):
  for doc in documents:
    # Apply basic cleaning
    cleaned_text = clean_text(doc.page_content)
    # Optionally remove stop words
    # cleaned_text = remove_stop_words(cleaned_text)  # Uncomment if you want stop word removal
    doc.page_content = cleaned_text
  return documents
pre_documents = preprocess_documents(documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, 
                                               chunk_overlap=64, 
                                               add_start_index=True,
                                               length_function = len,
                                               separators=["\n\n", "\n", "(?<=[\.?])", "(?<=[\,;])", " "])
# 正则表达式正向回顾后发断言 (?<=...) 分割.,?;
all_splits = text_splitter.split_documents(pre_documents)
all_splits

[Document(page_content='Retrieval-Augmented Generation for Large Language Models: A Survey\n\nYunfan Gao 1 , Yun Xiong 2 , Xinyu Gao 2 , Kangxiang Jia 2 , Jinliu Pan 2 , Yuxi Bi 3 , Yi\nDai1 , Jiawei Sun1 , Qianyu Guo4 , Meng Wang 3 and Haofen Wang 1,3 ∗\n1 Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University\n2 Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University\n3 College of Design and Innovation, Tongji University\n4 School of Computer Science, Fudan University\n\n4\n2\n0\n2', metadata={'source': '../documents/2312.10997.pdf', 'page': '0', 'start_index': 0}),
 Document(page_content='4\n2\n0\n2\n\nn\na\nJ\n\n5\n\n]\nL\nC\n.\ns\nc\n[\n\n4\nv\n7\n9\n9\n0\n1\n.\n2\n1\n3\n2\n:\nv\ni\nX\nr\na\n\nAbstract\n\n(RAG)\n\nuntraceable', metadata={'source': '../documents/2312.10997.pdf', 'page': '0', 'start_index': 502}),
 Document(page_content='Large Language Models (LLMs) demonstrate\nsignificant capabilities but face challenges suc

In [2]:
pdf_loader = PDFMinerLoader("../documents/2312.10997.pdf")
documents = pdf_loader.load()
def clean_text(text):
  #text = text.lower()  # Convert to lowercase
  #text = re.sub(r'[^a-z0-9\s-]', '', text)  # Remove non-alphanumeric characters (except space and dash)
  text = re.sub(r'-\n', '', text)  # Remove hyphens at line breaks
  return text
def preprocess_documents(documents):
  for doc in documents:
    # Apply basic cleaning
    cleaned_text = clean_text(doc.page_content)
    # Optionally remove stop words
    # cleaned_text = remove_stop_words(cleaned_text)  # Uncomment if you want stop word removal
    doc.page_content = cleaned_text
  return documents
pre_documents = preprocess_documents(documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, 
                                               chunk_overlap=64, 
                                               add_start_index=True,
                                               length_function = len,
                                               separators=["\n\n", "\n", "(?<=[\.?])", "(?<=[\,;])", " "])
# 正则表达式正向回顾后发断言 (?<=...) 分割.,?;
all_splits = text_splitter.split_documents(pre_documents)
all_splits

[Document(page_content='Retrieval-Augmented Generation for Large Language Models: A Survey\n\nYunfan Gao 1 , Yun Xiong 2 , Xinyu Gao 2 , Kangxiang Jia 2 , Jinliu Pan 2 , Yuxi Bi 3 , Yi\nDai1 , Jiawei Sun1 , Qianyu Guo4 , Meng Wang 3 and Haofen Wang 1,3 ∗\n1 Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University\n2 Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University\n3 College of Design and Innovation, Tongji University\n4 School of Computer Science, Fudan University\n\n4\n2\n0\n2', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 0}),
 Document(page_content='4\n2\n0\n2\n\nn\na\nJ\n\n5\n\n]\nL\nC\n.\ns\nc\n[\n\n4\nv\n7\n9\n9\n0\n1\n.\n2\n1\n3\n2\n:\nv\ni\nX\nr\na\n\nAbstract\n\n(RAG)\n\nuntraceable', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 502}),
 Document(page_content='Large Language Models (LLMs) demonstrate\nsignificant capabilities but face challenges such\nas hallucination, outda

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")
FAISSdb = FAISS.from_documents(all_splits, embeddings) 
FAISS_retriever=FAISSdb.as_retriever(search_type="similarity", search_kwargs={"k": 5})
bm25_retriever = BM25Retriever.from_documents(all_splits)
bm25_retriever.k = 5
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, FAISS_retriever], weights=[0.5, 0.5]
)

In [6]:
#FAISS
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small") 
vectordb = FAISS.from_documents(all_splits, embeddings) 
retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})


In [4]:
#ChromaDB
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small") 
vectordb = Chroma.from_documents(all_splits, embeddings) 
retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [10]:
ensemble_retriever.invoke("What is naive rag")

[Document(page_content='[Blagojevi, 2023] Vladimir Blagojevi.\n\npipelines in haystack:\nlostinthemiddleranker.\nenhancing-rag-pipelines-in-haystack-45f14e2bc9f5,\n2023.\n\nEnhancing rag\nIntroducing diversityranker and\nhttps://towardsdatascience.com/', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 101385}),
 Document(page_content='3.1 Naive RAG\nThe Naive RAG research paradigm represents the earliest\nmethodology, which gained prominence shortly after the\nwidespread adoption of ChatGPT. The Naive RAG follows a\ntraditional process that includes indexing, retrieval, and generation. It is also characterized as a “Retrieve-Read” framework [Ma et al., 2023a].', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 12575}),
 Document(page_content='upon the foundation of Naive RAG by adding “Rewrite” and\n“Rerank” modules. However, on the whole, modular RAG\nenjoys greater diversity and flexibility.', metadata={'source': '../documents/2312.10997.pdf', 'star

In [4]:
retriever.invoke("What is naive rag")

[Document(page_content='3.1 Naive RAG\nThe Naive RAG research paradigm represents the earliest\nmethodology, which gained prominence shortly after the\nwidespread adoption of ChatGPT. The Naive RAG follows a\ntraditional process that includes indexing, retrieval, and generation. It is also characterized as a “Retrieve-Read” framework [Ma et al., 2023a].', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 12575}),
 Document(page_content='upon the foundation of Naive RAG by adding “Rewrite” and\n“Rerank” modules. However, on the whole, modular RAG\nenjoys greater diversity and flexibility.', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 27158}),
 Document(page_content='Drawbacks in Naive RAG\nNaive RAG faces significant challenges in three key areas:\n“Retrieval,” “Generation,” and “Augmentation”.', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 14950}),
 Document(page_content='3 RAG Framework\nThe RAG research paradigm is continuous

In [5]:
retriever.invoke("What is naive rag")

[Document(page_content='3.1 Naive RAG\nThe Naive RAG research paradigm represents the earliest\nmethodology, which gained prominence shortly after the\nwidespread adoption of ChatGPT. The Naive RAG follows a\ntraditional process that includes indexing, retrieval, and generation. It is also characterized as a “Retrieve-Read” framework [Ma et al., 2023a].', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 12575}),
 Document(page_content='upon the foundation of Naive RAG by adding “Rewrite” and\n“Rerank” modules. However, on the whole, modular RAG\nenjoys greater diversity and flexibility.', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 27158}),
 Document(page_content='Drawbacks in Naive RAG\nNaive RAG faces significant challenges in three key areas:\n“Retrieval,” “Generation,” and “Augmentation”.', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 14950}),
 Document(page_content='3 RAG Framework\nThe RAG research paradigm is continuous

In [5]:
# cohere正在更新中,暂时会产生bug
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank #cohere 4.57
#from langchain_cohere import CohereRerank #cohere v5 have bug
from apikeys import Cohere_API
print(Cohere_API)
# Create the retriever
compressor = CohereRerank(cohere_api_key=Cohere_API, top_n=5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)


Z1Xo88ygTulcat9cXl7t4X8sCjELSWOmiwq2Vz3R


In [6]:
test_retrieval = compression_retriever.get_relevant_documents("What is naive rag? give me a introduction")

test_retrieval

[Document(page_content='3.1 Naive RAG\nThe Naive RAG research paradigm represents the earliest\nmethodology, which gained prominence shortly after the\nwidespread adoption of ChatGPT. The Naive RAG follows a\ntraditional process that includes indexing, retrieval, and generation. It is also characterized as a “Retrieve-Read” framework [Ma et al., 2023a].', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 12575, 'relevance_score': 0.99855626}),
 Document(page_content='Drawbacks in Naive RAG\nNaive RAG faces significant challenges in three key areas:\n“Retrieval,” “Generation,” and “Augmentation”.', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 14950, 'relevance_score': 0.97796774}),
 Document(page_content='upon the foundation of Naive RAG by adding “Rewrite” and\n“Rerank” modules. However, on the whole, modular RAG\nenjoys greater diversity and flexibility.', metadata={'source': '../documents/2312.10997.pdf', 'start_index': 27158, 'relevance_score': 0